In [3]:
push!( LOAD_PATH, "." ); # folder que continen los modulos. Correr desde ahí dentro.
#
using Z_auxiliaresBRW, Z_auxiliaresGEN
#
using JLD, StatsBase
#
BRWname = "C:\\Users\\Siby\\Desktop\\200819\\control_01.brw";
vars = brw_things( BRWname );

ζ = vars[ "Factor" ]; # variables para...
ο = vars[ "Offset" ]; # conversión de voltaje
Γ = OS( "windows" ); # establecer SO a usar...es para mi lap si?

WORKpath = string( split( BRWname, "." )[ 1 ], Γ, "Process" ); 

files = find_files( WORKpath, ".jld" );

In [10]:
for i = 13:size( files, 1 )
 
    data = load( files[ i ] )[ "data" ]; # carga el bin i
    data = ( ( data.*ζ ) .+ ο ); # conversion a volaje
    
    δςAll = [ ]; ςζAll = [ ]; # preallocation
 
    # Remover saturaciones positivas
    # using Statistics
    # thr = maximum([maximum(data),std(data)*4]);
    thr = 4000; # arbitriario, stfu
    Ωτ = findall( data .>= thr ); # canales y Xs que sobrepasan el threshold
    
    Θ = zeros( Int, size( Ωτ, 1 ), 2 ); # preallocation
    
    if isempty( Ωτ ) == false
        for j = 1:size( Ωτ, 1 )
            Θ[ j, 1 ] = Ωτ[ j ].I[ 1 ]; # channel
            Θ[ j, 2 ] = Ωτ[ j ].I[ 2 ]; # X
        end
    end
    
    #= obtener canales saturados para promediacion (Σ).
    Aquellos saturados durante todo el bin son descartados de la lista (δς)) =#
    Σ = zeros( Int, length( countmap( Θ[ :, 1 ] ) ), 2 ); # preallocation
    Σ[ :, 1 ] = Int.( keys( countmap( Θ[ :, 1 ] ) ) ); # que canales
    Σ[ :, 2 ] = Int.( values(countmap( Θ[ :, 1 ] ) ) ); # cuantas veces
    
    # mas del 50% de Xs...se descarta el canal (ς) de la lista de promediación
    δς = Σ[ Σ[ :, 2 ] .>= Int( floor( 0.50*size( data, 2 ) ) ), 1 ]; 
    push!( δςAll, δς );
    
    #= obtener Xs saturados para promediacion (Φ).
    Aquellos saturados durante todo el bin son descartados de la lista (ςζ)) =#
    Φ = zeros( Int, length( countmap( Θ[ :, 2 ] ) ), 2 ); # preallocation
    Φ[ :, 1 ] = Int.( keys( countmap( Θ[ :, 2 ] ) ) ); # que Xs
    Φ[ :, 2 ] = Int.( values( countmap( Θ[ :, 2 ] ) ) ); # cuantas veces
    ςζ = Φ[ Φ[ :, 2 ] .>= Int( floor( 0.50*size( data, 1 ) ) ), 1 ]; # mas del 50% de canales...3480
    push!( ςζAll, ςζ );
    
    #
    Θ = Θ[ Bool.( 1 .- in.( Θ[ :, 1 ], [ δς ] ) ), : ]; # final list of channels
    Θ = Θ[ Bool.( 1 .- in.( Θ[ :, 2 ], [ ςζ ] ) ), : ]; # final list of Xs
    #
    τ = collect( 1:1:size( data, 1 ) + 1 ); # canales que pueden ser usados para reparar saturaciones
    τ = τ[ Bool.( 1 .- in.( τ, [ δς ] ) ) ]; # without the all saturated channels

    χ = collect( 1:1:size( data, 2 ) ); # Xs que pueden ser usados para reparar saturaciones
    χ = χ[ Bool.( 1 .- in.( χ, [ ςζ ] ) ) ]; # without the all saturated Xs
    #
    
    for l = 1:size( Θ, 1 )
        ςμ = Θ[ l, 1 ]; # channel and
        X = Θ[ l, 2 ]; # X for correction
        σ1 = isempty( τ[ τ .== ςμ ] ) == false # if the channel is valid
        σ2 = isempty( χ[ χ .== X ] ) == false # and the X is valid
        if σ1 & σ2
            Ης = vec( reshape( neighborgs( ςμ, 1 ), length( neighborgs( ςμ, 1 ) ), 1 ) ); # neighborgs
            filter!( e -> e≠ςμ, Ης ) # without the center of course
            filter!( e -> e≠τ, Ης )
            ΗςX = zeros( Int, length( Ης ), 2 );
            ΗςX[ :, 1 ] = Ης;
            ΗςX[ :, 2 ] = repeat( [ X ], length( Ης ) );
            ΗςX = ΗςX[ Bool.( 1 .- in.( ΗςX[ :, 1 ], [ δς ] ) ), : ]; # final list of channels (neigh)
            ΗςX = ΗςX[ Bool.( 1 .- in.( ΗςX[ :, 2 ], [ ςζ ] ) ), : ]; # final list of Xs (neig)
            if isempty( ΗςX ) == false
                global data[ ςμ, X ] = mean( data[ ΗςX ][ :, 1 ] );
            end
        end
    end
    #
    println( "positive saturations removed." );
    #
    # Remover saturaciones negativas

    #using Statistics
    #thr = maximum([maximum(data),std(data)*4]);
    thr = -4000;
    υτ = findall(data .<= thr); # canales y Xs que sobrepasan el threshold
    if isempty(υτ) == false
        Υ = zeros(Int,size(υτ,1),2); #preallocation
        for j = 1:size(υτ,1)
            Υ[j,1] = υτ[j].I[1]; # channel
            Υ[j,2] = υτ[j].I[2]; # X
        end
    end
    #= obtener canales saturados para promediacion (Σ).
    Aquellos saturados durante todo el bin son descartados de la lista (δς)) =#
    Σ = zeros( Int, length( countmap( Υ[ :, 1 ] ) ), 2 ); # preallocation
    Σ[ :, 1 ] = Int.( keys( countmap( Υ[ :, 1 ] ) ) ); # que canales
    Σ[ :, 2 ] = Int.( values( countmap( Υ[ :, 1 ] ) ) ); # cuantas veces
    δς = Σ[ Σ[ :, 2 ] .>= Int( floor( 0.50*size( data, 2 ) ) ), 1 ]; # mas del 50% de Xs...42029
    push!( δςAll, δς );
    
    #= obtener Xs saturados para promediacion (Φ).
    Aquellos saturados durante todo el bin son descartados de la lista (ςζ)) =#
    Φ = zeros( Int, length( countmap( Υ[ :, 2 ] ) ), 2 ); # preallocation
    Φ[ :, 1 ] = Int.( keys( countmap( Υ[ :, 2 ] ) ) ); # que Xs
    Φ[ :, 2 ] = Int.( values( countmap( Υ[ :, 2 ] ) ) ); # cuantas veces
    ςζ = Φ[ Φ[ :, 2 ] .>= Int( floor( 0.50*size( data, 1 ) ) ), 1 ]; # mas del 50% de canales...3480
    push!( ςζAll, ςζ );
    #
    Υ = Υ[ Bool.( 1 .- in.( Υ[ :, 1 ], [ δς ] ) ), : ]; # final list of channels
    Υ = Υ[ Bool.( 1 .- in.( Υ[ :, 2 ], [ ςζ ] ) ), : ]; # final list of Xs
    #
    τ = collect( 1:( size( data, 1 ) + 1 ) ); # canales que pueden ser usados para reparar saturaciones
    τ = τ[ Bool.( 1 .- in.( τ, [ δς ] ) ) ]; # without the all saturated channels

    χ = collect( 1:size( data, 2 ) ); # Xs que pueden ser usados para reparar saturaciones
    χ = χ[ Bool.( 1 .- in.( χ, [ ςζ ] ) ) ]; # without the all saturated Xs
    #
    for l = 1:size( Υ, 1 )
        ςμ = Υ[ l, 1 ]; # channel and
        X = Υ[ l, 2 ]; # frames for correction
        σ1 = isempty( τ[ τ .== ςμ ] ) == false # if the channel is valid
        σ2 = isempty( χ[ χ .== X ] ) == false # and the frame is valid
        if σ1 & σ2
            Ης = vec( reshape( neighborgs( ςμ, 1 ), length( neighborgs( ςμ, 1 ) ), 1 ) ); # neighborgs
            filter!( e -> e≠ςμ, Ης ) # without the center of course
            filter!( e-> e≠τ, Ης )
            ΗςX = zeros( Int, length( Ης ), 2 );
            ΗςX[ :, 1 ] = Ης;
            ΗςX[ :, 2 ] = repeat( [ X ], length( Ης ) );
            ΗςX = ΗςX[ Bool.( 1 .- in.( ΗςX[ :, 1 ], [ δς ] ) ), : ]; # final list of channels (neigh)
            ΗςX = ΗςX[ Bool.( 1 .- in.( ΗςX[ :, 2 ], [ ςζ ] ) ), : ]; # final list of Xs (neig)
            if isempty( ΗςX ) == false
                global data[ ςμ, X ] = mean( data[ ΗςX ][ :, 1 ] );
            end
        end
    end
    #
    println( "negative saturations removed." );
    #
    δςAll = vcat( δςAll[ 1 ], δςAll[ 2 ] );
    ςζAll = vcat( ςζAll[ 1 ], ςζAll[ 2 ] );
    
    data[ δςAll, : ] .= 0; data[ :, ςζAll ] .= 0;
    
    save( files[ i ], "data", data );
    println( string( "file: ",i," of ", size( files, 1 ), " saved without saturations." ) );
end

BoundsError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]